In [35]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from datasets import load_metric,load_dataset,Dataset

import transformers
from transformers import AutoTokenizer, DataCollatorWithPadding,RobertaForSequenceClassification,AdamW,get_scheduler,TrainingArguments,Trainer
from corpy.morphodita import Tokenizer
from newspaper import Article

import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,StratifiedKFold
from tqdm.auto import tqdm, trange

import csv
import gc
import re

model_checkpoint = 'ufal/robeczech-base'

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
transformers.logging.set_verbosity(transformers.logging.ERROR)

BATCH_SIZE = 32

In [36]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


def clean_memory():
    gc.collect()
    torch.cuda.empty_cache()
    
def compute_metrics(testing_dataloader):
    metric = load_metric("f1")
    metric2 = load_metric("accuracy")

    model.eval()
    for batch in testing_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])
        metric2.add_batch(predictions=predictions, references=batch["labels"])

        
    return (metric.compute(average='micro'),metric2.compute())

## Data preprocessing

In [37]:
data = pd.read_csv('../data/BABE/final_labels_SG2.csv',sep=';')
data = data[['text','label_bias']]
final_indices = data.index[data['label_bias'] != 'No agreement'].tolist()
data = data[data['label_bias']!='No agreement']

mapping = {'Non-biased':0, 'Biased':1}
data.replace({'label_bias':mapping},inplace=True)
data_en = data

In [38]:
with open('../data/BABE/texts_CS.txt','r') as f:
    sentences = [sentence.strip('\n') for sentence in f.readlines()]
    sentences = list(filter(lambda x: len(x) != 0, sentences))

In [39]:
sentences = np.array(sentences)[final_indices]

In [40]:
data = Dataset.from_dict({'sentence':sentences,'label':data['label_bias']})

## Training

In [41]:
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [42]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=False) #fast tokenizer is buggy in RoBERTa models
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint)
model.to(device);

In [43]:
tokenize = lambda data : tokenizer(data['sentence'], truncation=True)

In [44]:
tokenized_data = data.map(tokenize,batched=True)
tokenized_data = tokenized_data.remove_columns(['sentence'])
tokenized_data.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [45]:
training_args = TrainingArguments(
    output_dir='../',
    num_train_epochs=10,
    per_device_train_batch_size=BATCH_SIZE,  
    logging_steps=25,
    disable_tqdm = False,
    save_total_limit=2,
    learning_rate=5e-5)

### 5-fold CV

In [109]:
scores = []

In [110]:
for train_index, val_index in skfold.split(tokenized_data['input_ids'],tokenized_data['label']):
    
    token_train = Dataset.from_dict(tokenized_data[train_index])
    token_valid = Dataset.from_dict(tokenized_data[val_index])
    
    model = RobertaForSequenceClassification.from_pretrained(model_checkpoint);
    trainer = Trainer(model,training_args,train_dataset=token_train,data_collator=data_collator,
                      tokenizer=tokenizer)
    trainer.train()
    
    #evaluation
    eval_dataloader = DataLoader(token_valid, batch_size=BATCH_SIZE, collate_fn=data_collator)
    scores.append(compute_metrics(eval_dataloader))


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/torch_formatter.py:44: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:135.)
  return torch.tensor(value, **{**default_dtyp

Step,Training Loss
25,0.619100
50,0.551800
75,0.551100
100,0.515900
125,0.445800
150,0.432100
175,0.398400
200,0.352800
225,0.301300
250,0.280100


Saving model checkpoint to ../checkpoint-500
Configuration saved in ../checkpoint-500/config.json
Model weights saved in ../checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../checkpoint-500/tokenizer_config.json
Special tokens file saved in ../checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb

Step,Training Loss
25,0.647000
50,0.559100
75,0.552500
100,0.546900
125,0.451600
150,0.450900
175,0.424900
200,0.381200
225,0.299600
250,0.295500


Saving model checkpoint to ../checkpoint-500
Configuration saved in ../checkpoint-500/config.json
Model weights saved in ../checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../checkpoint-500/tokenizer_config.json
Special tokens file saved in ../checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb

...

Saving model checkpoint to ../checkpoint-500
Configuration saved in ../checkpoint-500/config.json
Model weights saved in ../checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../checkpoint-500/tokenizer_config.json
Special tokens file saved in ../checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb

Step,Training Loss
25,0.645500
50,0.552600
75,0.500800
100,0.495600
125,0.479200
150,0.401200
175,0.402200
200,0.362400
225,0.295900
250,0.305200


Saving model checkpoint to ../checkpoint-500
Configuration saved in ../checkpoint-500/config.json
Model weights saved in ../checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../checkpoint-500/tokenizer_config.json
Special tokens file saved in ../checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


/home/horyctom/venv/lib/python3.8/site-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)
loading configuration file https://huggingface.co/ufal/robeczech-base/resolve/main/config.json from cache at /home/horyctom/.cache/huggingface/transformers/967e55aeea0667ffcda38959128e06f755d387fa034ffb

Step,Training Loss
25,0.643000
50,0.538500
75,0.520000
100,0.496700
125,0.447900
150,0.386100
175,0.423300
200,0.311100
225,0.329500
250,0.282800


Saving model checkpoint to ../checkpoint-500
Configuration saved in ../checkpoint-500/config.json
Model weights saved in ../checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../checkpoint-500/tokenizer_config.json
Special tokens file saved in ../checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




In [111]:
#torch.save(model.state_dict(),'../cs_babe.pth')
model.load_state_dict(torch.load('../cs_babe.pth'))
model.eval();


In [117]:
scores

[({'f1': 0.782312925170068}, {'accuracy': 0.782312925170068}),
 ({'f1': 0.7945578231292517}, {'accuracy': 0.7945578231292517}),
 ({'f1': 0.7904761904761904}, {'accuracy': 0.7904761904761904}),
 ({'f1': 0.7656675749318801}, {'accuracy': 0.7656675749318801}),
 ({'f1': 0.7697547683923706}, {'accuracy': 0.7697547683923706})]

## Training on full data 

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_checkpoint);
trainer = Trainer(model,training_args,train_dataset=tokenized_data,data_collator=data_collator,
                      tokenizer=tokenizer)
trainer.train()

***** Running training *****
  Num examples = 3673
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1150


Step,Training Loss
25,0.640600
50,0.546600
75,0.516700
100,0.556500
125,0.479700
150,0.445900
175,0.405400
200,0.412300
225,0.441100
250,0.331100


Saving model checkpoint to ../checkpoint-500
Configuration saved in ../checkpoint-500/config.json
Model weights saved in ../checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../checkpoint-500/tokenizer_config.json
Special tokens file saved in ../checkpoint-500/special_tokens_map.json


In [8]:
torch.save(model.state_dict(),'../cs_babe.pth')
model.load_state_dict(torch.load('../cs_babe.pth'))
model.eval();

# Inferrence experiments

In [5]:
def classify_sentence(sent:str):
    toksentence = tokenizer(sent,truncation=True,return_tensors="pt")
    model.eval()
    with torch.no_grad():
        toksentence.to(device)
        output = model(**toksentence)
    
    classification = F.softmax(output.logits,dim=1).argmax(dim=1)
    
    return {0:'unbiased',1:'biased'}[classification[0].item()]

### Nice pairs

In [185]:
print(classify_sentence("Od posledního Štědrýho večera a následující noci na mě máma ani jednou nepromluvila."))
print(classify_sentence("Od posledního Štědrýho večera a následující noci na mě guru máma ani jednou nepromluvila."))

unbiased
biased


In [187]:
print(classify_sentence('Podle íránské vlády bylo sestřelení IR 655 "Vincennes" úmyslně provedeným a nezákonným činem.'))
print(classify_sentence('Sestřelení IR 655 "Vincennes" bylo úmyslně provedeným a nezákonným činem.'))

unbiased
biased


### CW-hard biased data

In [190]:
with open("biased_cwhard_cs.txt","r") as f:
    cw_hard_cs = f.read().splitlines()

In [191]:
annotations = np.array(list(zip(cw_hard_cs,list(map(classify_sentence,cw_hard_cs)))))

In [195]:
stats = np.unique(annotations[:,1],return_counts=True)
print("bias level: ",stats[1][0]/stats[1].sum()*100,"%")

bias level:  32.44709712425393 %


### Try on any article

In [163]:
article = Article('https://nazory.aktualne.cz/komentare/jak-jsem-na-stedrej-den-zradila-nasi-antivax-familiji/r~93a757c460d211eca1070cc47ab5f122/')
article.download()
article.parse()

tokenizer_morphodita = Tokenizer("czech")

all = []
for sentence in tokenizer_morphodita.tokenize(article.text, sents=True):
    all.append(sentence)
    
sentences = np.array([' '.join(x) for x in all])
annotations = np.array(list(zip(sentences,list(map(classify_sentence,sentences)))))
stats = np.unique(annotations[:,1],return_counts=True)

print("bias level: ",stats[1][0]/stats[1].sum()*100,"%")

bias level:  52.32974910394266 %


In [165]:
list(filter(lambda x : x[1] == 'biased',annotations))

[array(['Od posledního Štědrýho večera a následující noci na mě guru máma ani jednou nepromluvila .',
        'biased'], dtype='<U588'),
 array(['" Zrádce národa , " vykřikla máma a bouchla dlaní do volantu , " zrádce zasranej " .',
        'biased'], dtype='<U588'),
 array(['Jo , abych nezapomněla , ono to sem patří , zrádcem národa je nějakej epidemiolog , jméno mi uteklo , kterej nabádá , ať se lidi očkujou .',
        'biased'], dtype='<U588'),
 array(['On je typickej podpantoflák , ale poslední dobou se taky maličko změnil a občas aspoň pípne , asi už je toho i na něj trochu moc .',
        'biased'], dtype='<U588'),
 array(['Jemu neodpověděla , ale otočila se na mě dozadu , přitom jela na úzký silnici přes osmdesát , a povídá : " Chápej , Mončo , " ježíšmarjá , jak já nenávidím to svý jméno , Monika , ale k úplnýmu šílenství mě dohání Monča , Mončičák a Mončičáček , vraždila bych kvůli tomu , " neříkej mi Mončo , prosím , " zaúpěla jsem a ona sadisticky , " chápej , Mončičáčku , 